In [1]:
import plaidml.keras
plaidml.keras.install_backend()
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

In [2]:
# Importing useful libraries
import numpy as np
import matplotlib.pyplot as plt
import keras
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional, Conv1D, Flatten, MaxPooling1D
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras import optimizers

import time 

In [3]:
# from google.colab import files
# uploaded = files.upload()

In [4]:
df = pd.read_csv('../data/num_data.csv')

In [5]:
keras.backend.clear_session()


### Data Processing

In [6]:
# df = pd.read_csv('../data/num_data.csv')

In [7]:
POLLUTION = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3']

In [8]:
WEATHER = ['PM2.5', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'wd', 'WSPM']

In [9]:
dataset = df

In [10]:
dataset.shape

(420768, 16)

In [11]:
# Useful functions
def plot_predictions(test, predicted):
    plt.figure(figsize=(30, 15));

    plt.plot(test, color='red', alpha=0.5, label='Actual PM2.5 Concentration',)
    plt.plot(predicted, color='blue', alpha=0.5, label='Predicted PM2.5 Concentation')
    plt.title('PM2.5 Concentration Prediction')
    plt.xlabel('Time')
    plt.ylabel('PM2.5  Concentration')
    plt.legend()
    plt.show()
    

def return_rmse(test,predicted):
    rmse = math.sqrt(mean_squared_error(test, predicted))
    return rmse

In [12]:
data_size = dataset.shape[0]
train_size=int(data_size * 0.6)
test_size = int(data_size * 0.2)
valid_size = data_size - train_size - test_size


In [13]:
training_set = dataset[:train_size].iloc[:,4:16].values
valid_set = dataset[train_size:train_size+valid_size].iloc[:,4:16].values
test_set = dataset[data_size-test_size:].iloc[:,4:16].values

In [14]:
y = dataset.iloc[:,0].values
y = y.reshape(-1,1)
n_feature = training_set.shape[1]
y.shape

(420768, 1)

In [15]:
# Scaling the dataset
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)
valid_set_scaled = sc.fit_transform(valid_set)
test_set_scaled = sc.fit_transform(test_set)

sc_y = MinMaxScaler(feature_range=(0,1))
y_scaled = sc_y.fit_transform(y)

In [16]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X_, y_ = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix-1:out_end_ix, 0]
        X_.append(seq_x)
        y_.append(seq_y)
    return np.array(X_), np.array(y_)

In [17]:
n_steps_in = 12
n_steps_out = 12
X_train, y_train = split_sequences(training_set_scaled, n_steps_in, n_steps_out)
X_valid, y_valid = split_sequences(valid_set_scaled, n_steps_in, n_steps_out)
X_test, y_test = split_sequences(test_set_scaled, n_steps_in, n_steps_out)

## Grid Search Control 


In [18]:
n_activation = ['tanh', 'sigmoid', 'relu']
act = n_activation[0]

n_learn_rate = [0.01, 0.1, 0.2]
lr = n_learn_rate[0]

n_optimizers = [optimizers.Adam(lr=lr), optimizers.RMSprop(lr=lr), optimizers.SGD(lr=lr)]
opt = n_optimizers[0]

n_epoches = [50]
epoch = n_epoches[0]

n_batch_size = [32, 256, 1024]
batch = n_batch_size[-1]

n_of_neurons = [10, 50, 200]
neuron = n_of_neurons[1]

INFO:plaidml:Opening device "llvm_cpu.0"


In [19]:
rmse_df = pd.DataFrame(columns=['Model', 'train_rmse', 'valid_rmse', 'test_rmse', 'train_time', 'epoch', 
                               'batch', 'neuron'])

In [20]:
DFS = Sequential()


In [21]:
for act in n_activation:
    for lr in n_learn_rate:
        n_optimizers = [optimizers.Adam(lr=lr), optimizers.RMSprop(lr=lr), optimizers.SGD(lr=lr)]
        for opt in n_optimizers:        
            DFS = Sequential()
            DFS.add(Conv1D(filters=64, kernel_size=6, activation='tanh', input_shape=(X_train.shape[1],n_feature)))
            DFS.add(MaxPooling1D(pool_size=4))
            DFS.add(Dropout(0.2))  
            DFS.add(LSTM(units=neuron, return_sequences=False, input_shape=(X_train.shape[1],n_feature), activation=act))
            DFS.add(Dropout(0.190 + 0.0025 * n_steps_in))
            DFS.add(Dense(units=n_steps_out))
            DFS.compile(optimizer=opt,loss='mean_squared_error')

            
            regressor = DFS
            model = 'DFS'
    
            print('training start for', model)    
            start = time.process_time()
            regressor.fit(X_train,y_train,epochs=epoch,batch_size=batch)
            train_time = round(time.process_time() - start, 2)

            print('results for training set')
            y_train_pred = regressor.predict(X_train)
            train_rmse = return_rmse(y_train,y_train_pred)

            print('results for valid set')
            y_valid_pred = regressor.predict(X_valid)
            valid_rmse = return_rmse(y_valid,y_valid_pred)    
            
            
            print('results for test set')
            y_test_pred = regressor.predict(X_test)
            test_rmse = return_rmse(y_test,y_test_pred)
    
            one_df = pd.DataFrame([[model, train_rmse, valid_rmse, test_rmse, train_time, epoch, batch, neuron]],
                          columns=['Model', 'train_rmse', 'valid_rmse', 'test_rmse', 'train_time', 'epoch', 
                               'batch', 'neuron'])
            rmse_df = pd.concat([rmse_df, one_df])

# save the rmse results 
rmse_df.to_csv('../dfs_grid_search_v2.csv')
          

training start for DFS
Epoch 1/50
252438/252438 [==============================] - 22s 89us/step - loss: 0.0039
Epoch 2/50
252438/252438 [==============================] - 18s 73us/step - loss: 0.0034
Epoch 3/50
252438/252438 [==============================] - 24s 93us/step - loss: 0.0034
Epoch 4/50
252438/252438 [==============================] - 22s 88us/step - loss: 0.0033
Epoch 5/50
252438/252438 [==============================] - 22s 86us/step - loss: 0.0033
Epoch 6/50
252438/252438 [==============================] - 22s 86us/step - loss: 0.0032
Epoch 7/50
252438/252438 [==============================] - 22s 85us/step - loss: 0.0032
Epoch 8/50
252438/252438 [==============================] - 22s 86us/step - loss: 0.0032
Epoch 9/50
252438/252438 [==============================] - 22s 87us/step - loss: 0.0032
Epoch 10/50
252438/252438 [==============================] - 22s 86us/step - loss: 0.0031
Epoch 11/50
252438/252438 [==============================] - 22s 89us/step - loss: 0.0

252438/252438 [==============================] - 27s 108us/step - loss: 0.0073
Epoch 28/50
252438/252438 [==============================] - 27s 107us/step - loss: 0.0072
Epoch 29/50
252438/252438 [==============================] - 28s 109us/step - loss: 0.0073
Epoch 30/50
252438/252438 [==============================] - 27s 109us/step - loss: 0.0072
Epoch 31/50
252438/252438 [==============================] - 26s 103us/step - loss: 0.0073
Epoch 32/50
252438/252438 [==============================] - 28s 111us/step - loss: 0.0074
Epoch 33/50
252438/252438 [==============================] - 27s 107us/step - loss: 0.0072
Epoch 34/50
252438/252438 [==============================] - 27s 106us/step - loss: 0.0072
Epoch 35/50
252438/252438 [==============================] - 24s 96us/step - loss: 0.0073
Epoch 36/50
252438/252438 [==============================] - 24s 94us/step - loss: 0.0061
Epoch 37/50
252438/252438 [==============================] - 25s 98us/step - loss: 0.0060
Epoch 38/50
25

252438/252438 [==============================] - 24s 94us/step - loss: 1.3888
Epoch 4/50
252438/252438 [==============================] - 24s 95us/step - loss: 1.3589 1s 
Epoch 5/50
252438/252438 [==============================] - 24s 96us/step - loss: 1.1890
Epoch 6/50
252438/252438 [==============================] - 25s 98us/step - loss: 0.8871
Epoch 7/50
252438/252438 [==============================] - 24s 95us/step - loss: 0.7666
Epoch 8/50
252438/252438 [==============================] - 24s 95us/step - loss: 0.7521 0s - los
Epoch 9/50
252438/252438 [==============================] - 24s 95us/step - loss: 0.6392
Epoch 10/50
252438/252438 [==============================] - 25s 98us/step - loss: 0.4648 0s - lo
Epoch 11/50
252438/252438 [==============================] - 24s 94us/step - loss: 0.4197
Epoch 12/50
252438/252438 [==============================] - 24s 94us/step - loss: 0.4184
Epoch 13/50
252438/252438 [==============================] - 24s 96us/step - loss: 0.4198
Epoch 1

252438/252438 [==============================] - 29s 116us/step - loss: 0.0036
Epoch 30/50
252438/252438 [==============================] - 29s 116us/step - loss: 0.0036
Epoch 31/50
252438/252438 [==============================] - 29s 116us/step - loss: 0.0036
Epoch 32/50
252438/252438 [==============================] - 31s 122us/step - loss: 0.0036
Epoch 33/50
252438/252438 [==============================] - 24s 95us/step - loss: 0.0036 8s - l
Epoch 34/50
252438/252438 [==============================] - 26s 103us/step - loss: 0.0036
Epoch 35/50
252438/252438 [==============================] - 32s 126us/step - loss: 0.0036
Epoch 36/50
252438/252438 [==============================] - 29s 116us/step - loss: 0.00360s - loss: 0.003
Epoch 37/50
252438/252438 [==============================] - 31s 124us/step - loss: 0.0036
Epoch 38/50
252438/252438 [==============================] - 29s 113us/step - loss: 0.0036
Epoch 39/50
252438/252438 [==============================] - 34s 136us/step - lo

252438/252438 [==============================] - 33s 131us/step - loss: 0.0229
Epoch 5/50
252438/252438 [==============================] - 33s 130us/step - loss: 0.0198
Epoch 6/50
252438/252438 [==============================] - 27s 106us/step - loss: 0.01740s - loss:
Epoch 7/50
252438/252438 [==============================] - 23s 91us/step - loss: 0.0155
Epoch 8/50
252438/252438 [==============================] - 23s 91us/step - loss: 0.0141
Epoch 9/50
252438/252438 [==============================] - 22s 87us/step - loss: 0.0129
Epoch 10/50
252438/252438 [==============================] - 24s 95us/step - loss: 0.0120 1s - loss: 0 - ETA: 0s - loss: 
Epoch 11/50
252438/252438 [==============================] - 24s 95us/step - loss: 0.0112
Epoch 12/50
252438/252438 [==============================] - 23s 92us/step - loss: 0.0106
Epoch 13/50
252438/252438 [==============================] - 23s 90us/step - loss: 0.0101
Epoch 14/50
252438/252438 [==============================] - 24s 95us/st

252438/252438 [==============================] - 23s 90us/step - loss: 0.0055
Epoch 32/50
252438/252438 [==============================] - 22s 86us/step - loss: 0.0054
Epoch 33/50
252438/252438 [==============================] - 22s 87us/step - loss: 0.0053
Epoch 34/50
252438/252438 [==============================] - 23s 90us/step - loss: 0.0052
Epoch 35/50
252438/252438 [==============================] - 22s 89us/step - loss: 0.0052
Epoch 36/50
252438/252438 [==============================] - 21s 84us/step - loss: 0.0051
Epoch 37/50
252438/252438 [==============================] - 22s 86us/step - loss: 0.0050
Epoch 38/50
252438/252438 [==============================] - 23s 89us/step - loss: 0.0050
Epoch 39/50
252438/252438 [==============================] - 21s 85us/step - loss: 0.0049
Epoch 40/50
252438/252438 [==============================] - 22s 88us/step - loss: 0.0049
Epoch 41/50
252438/252438 [==============================] - 21s 84us/step - loss: 0.0049
Epoch 42/50
252438/252

252438/252438 [==============================] - 22s 88us/step - loss: 0.0069
Epoch 9/50
252438/252438 [==============================] - 22s 87us/step - loss: 0.0069
Epoch 10/50
252438/252438 [==============================] - 22s 88us/step - loss: 0.0069
Epoch 11/50
252438/252438 [==============================] - 23s 90us/step - loss: 0.0069
Epoch 12/50
252438/252438 [==============================] - 23s 91us/step - loss: 0.0069
Epoch 13/50
252438/252438 [==============================] - 22s 86us/step - loss: 0.0069
Epoch 14/50
252438/252438 [==============================] - 22s 87us/step - loss: 0.0069
Epoch 15/50
252438/252438 [==============================] - 22s 88us/step - loss: 0.0069
Epoch 16/50
252438/252438 [==============================] - 22s 88us/step - loss: 0.0069
Epoch 17/50
252438/252438 [==============================] - 22s 87us/step - loss: 0.0069
Epoch 18/50
252438/252438 [==============================] - 22s 89us/step - loss: 0.0069
Epoch 19/50
252438/2524

252438/252438 [==============================] - 23s 91us/step - loss: 0.0070 0s - loss: 0.007 - ETA: 0s - loss: 0.
Epoch 35/50
252438/252438 [==============================] - 23s 91us/step - loss: 0.0069
Epoch 36/50
252438/252438 [==============================] - 21s 83us/step - loss: 0.0070
Epoch 37/50
252438/252438 [==============================] - 16s 62us/step - loss: 0.0070
Epoch 38/50
252438/252438 [==============================] - 15s 60us/step - loss: 0.0070
Epoch 39/50
252438/252438 [==============================] - 18s 70us/step - loss: 0.0070
Epoch 40/50
252438/252438 [==============================] - 23s 90us/step - loss: 0.0070
Epoch 41/50
252438/252438 [==============================] - 21s 83us/step - loss: 0.0069
Epoch 42/50
252438/252438 [==============================] - 23s 93us/step - loss: 0.0070
Epoch 43/50
252438/252438 [==============================] - 23s 90us/step - loss: 0.0070 1
Epoch 44/50
252438/252438 [==============================] - 22s 89us/st

In [22]:
rmse_df

,Model,train_rmse,valid_rmse,test_rmse,train_time,epoch,batch,neuron
0,DFS,0.051040,0.054292,0.056263,6160.77,50,1024,50
0,DFS,0.052712,0.056035,0.056691,7416.15,50,1024,50
0,DFS,0.075659,0.077065,0.076825,7380.83,50,1024,50
0,DFS,0.076707,0.081101,0.080801,7266.63,50,1024,50
0,DFS,0.503611,0.504280,0.500533,7025.96,50,1024,50
0,DFS,0.062919,0.065156,0.063771,6929.07,50,1024,50
0,DFS,0.084326,0.085606,0.086045,6591.09,50,1024,50
0,DFS,0.617946,0.617483,0.622893,7177.48,50,1024,50
0,DFS,0.060775,0.063049,0.061737,7192.03,50,1024,50
0,DFS,0.053872,0.056895,0.056626,7484.88,50,1024,50
